In [ ]:
# Cells 4 onwards produce different outputs. Only run the cell(s) for the output(s) you want.

# this version only takes every nth post

In [35]:
import pandas as pd
import csv
import re
import string
import os
import nltk
import numpy as np
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\OMars\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
pip install -r cluster-requirements.txt  #might be some missing, let me know if so

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#might'


In [177]:
#change filepath to wherever you've stored csv.  Needs fields 'twitter.tweet/text' and 'twitter.tweet/created'


file = r'C:\Users\OMars\Documents\CASM_offline\Citizens-Advice\post-issues-w-stakeholders-for-cluster.csv'
file = str(file)
#print(file)
#df = pd.read_csv(file)


row_count = 0

for row in open(r'C:\Users\OMars\Documents\CASM_offline\Citizens-Advice\post-issues-w-stakeholders-for-cluster.csv', "r",encoding='utf-8'):
    row_count += 1

print(row_count)
    
n = input('Sample every nth cell, where n = ')
n = int(n)

#create pandas dataframe from only every nth row from csv - taken from https://stackoverflow.com/questions/53812094/select-every-nth-row-as-a-pandas-dataframe-without-reading-the-entire-file

skip = np.arange(row_count)
skip = np.delete(skip, np.arange(0, row_count, n))
df = pd.read_csv(file, skiprows = skip)
df.index.name = 'id'
print('df created')

#df = df.iloc[::n, :]

516836
Sample every nth cell, where n = 1
df created


C:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (11,15,21,27,29,41,53,59) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [48]:
#OPTION 1 Produces one file with one header

output = input('filename output? ')
output = output + '.txt'

a = list(df['twitter.tweet/text'])
b = '\n'.join(str(t) for t in a)

b = re.sub(r'[^a-zA-Z \n]+','', b)
#b = os.linesep.join([s for s in b.splitlines() if s])
#print(b)


with open(output, 'w', encoding = 'utf-8') as f:
    
    f.write('**** *all_text\n\n')
    f.write(b.lower())



filename output? 10000


In [12]:
#option 2 : produce one file with separated years

df['date'] = df['twitter.tweet/created'].str[:4]

with open('post-cluster-test.txt', 'w', encoding = 'utf-8') as f:

    for year in range (2017, 2022):
        year_str = str(year) 
        f.write('**** *year_'+year_str+'\n\n')
        df_temp = df[df['date'] == year]
        text = list(df['twitter.tweet/text'])
        text = '\n'.join(str(t) for t in text)
        text = re.sub(r'[^a-zA-Z \n]+','', text)
        f.write(text.lower())
        f.write('\n\n')

In [49]:
#option 3 : produce separate file for each year

df_1 = pd.read_csv(r'C:\Users\OMars\Dropbox\Oliver Sharing Stuff\2021 Work\CASM\Citizens-Advice\post-clean-for-clustering.csv')
df['date'] = df['twitter.tweet/created'].str[:4]


for year in range (2017, 2022):
    year_str = str(year)
    doc_title = 'post-cluster-'+year_str+'-100k-only-1.txt'
    with open(doc_title, 'w', encoding = 'utf-8') as f:
        f.write('**** *year_'+year_str+'\n\n')
        df_temp = df[df['date'] == year]
        text = list(df['twitter.tweet/text'])
        text = '\n'.join(str(t) for t in text)
        text = re.sub(r'[^a-zA-Z \n]+','', text)
        f.write(text.lower())

In [56]:
#option 4: one file with separated years and stopwords removed

#define stopword remover
# NLTK Stop words

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

df['date'] = df'twitter.tweet/created'].str[:4]

with open('post-cluster-all-years-nostops-1.txt', 'w', encoding = 'utf-8') as f:

    for year in range (2017, 2022):
        year_str = str(year) 
        f.write('**** *year_'+year_str+'\n\n')
        df_temp = df[df['date'] == year]
        text = list(df['twitter.tweet/text'])
        text = [word for word in text if word not in stopwords.words('english')]
        text = '\n'.join(str(t) for t in text)
        text = re.sub(r'[^a-zA-Z \n]+','', text)
        f.write(text.lower())
        f.write('\n\n')

In [178]:
#option 5: split by companies

companies = ('APC', 'Amazon', 'Collect-Plus', 'DHL', 'DPD', 'DX', 'Doddle', 'FedEx', 'Hermes', 'HubBox', 'InPost', 'Interparcel', 'P4D', 'Parcel-Broker', 'Parcel-Monkey', 'Parcel2Go', 'ParcelCompare', 'ParcelHero', 'Parcelforce', 'Parcelly', 'Pass-My-Parcel', 'Post-Office', 'Royal-Mail', 'TNT', 'Tuffnells', 'UK-Mail', 'UPS', 'Yodel', 'iPostParcels')

#create a new dataframe which only has matches plus text

with open('post-cluster-all-companies-all.txt', 'w', encoding = 'utf-8') as f:   

    for company in companies:

        thestring = company+'-matches'
        df_matches = df.loc[:,df.columns.str.endswith(thestring)]
        df_2 = pd.concat([df["twitter.tweet/text"], df_matches], axis=1)
        df_2.rename(columns = {list(df_2)[1]: 'match-column'}, inplace = True)
        df_3 = df_2[~df_2['match-column'].isnull()]
        print(company)
        print(df_3.head())
        f.write('**** *company_'+company.lower()+'\n\n')
        text = list(df_3['twitter.tweet/text'])
        text = '\n'.join(str(t) for t in text)
        text = re.sub(r'[^a-zA-Z \n ]+','', text)
        f.write(text.lower())
        f.write('\n\n')




APC
                                     twitter.tweet/text     match-column
id                                                                      
684   @APCOvernight I’ve got a parcel missing, I’ve ...  [@apcovernight]
3115  @APCcustserv could someone DM me please lookin...   [@apccustserv]
3196  Dear @APCOvernight - no matter how many times ...  [@apcovernight]
3466  @Etsy I have been repeatedly ignored by a sell...  [@apcovernight]
5215  @APCOvernight Have been awaiting a delivery fr...  [@apcovernight]
Amazon
                                   twitter.tweet/text match-column
id                                                                
16  @jasonalba With all the negativity in the worl...  [@amazonuk]
23  @winding_sios @AmazonUK Amazon playing it safe...  [@amazonuk]
42  @NikoSarcevic @AmazonUK Thanks! Not really hav...  [@amazonuk]
59  @PatrickMooney @AmazonHelp @AmazonUK They shou...  [@amazonuk]
80  @LimeyPrintmaker @AmazonUK @CalendarClubUK Yea...  [@amazonuk]
Collect-P

Parcelforce
                                    twitter.tweet/text    match-column
id                                                                    
1    @parcelforce been trying to change delivery ad...  [@parcelforce]
30   @parcelforce So I had a parcel sent yesterday ...  [@parcelforce]
79   @parcelforce Hello sent you a DM but still hav...  [@parcelforce]
106  @parcelforce It really is no wonder you are al...  [@parcelforce]
134  Actually, on checking the tracking code using ...  [@parcelforce]
Parcelly
                                       twitter.tweet/text match-column
id                                                                    
8252    @parcelly are you guys seriously lying about y...  [@parcelly]
65028   @parcelly @atmosfair The app says it’s invalua...  [@parcelly]
68770   @parcelly what is this and why does it happen ...  [@parcelly]
125622  'We already see the emergence of mandatory cha...  [@parcelly]
126878  We have some exciting news to announce! @parce..